# Bagian 1: Pemuatan dan Pra-pemrosesan Data

In [1]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
import re
import joblib
import shap
import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, average_precision_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE

print("Semua library berhasil di-import.")

Semua library berhasil di-import.


In [3]:
def clean_col_names(df):
    """Membersihkan nama kolom menjadi format yang lebih sederhana."""
    cols = df.columns
    new_cols = [re.sub(r'[^A-Za-z0-9]+', '', col) for col in cols]
    df.columns = new_cols
    return df

# 1. Muat data mentah
df_raw = pd.read_csv('Dataset_Kredit_Diperkaya.csv')
df = df_raw.copy()

# 2. Lakukan pra-pemrosesan pada kolom asli (dengan underscore)
print("Melakukan pra-pemrosesan pada kolom mata uang...")
currency_cols = ['Jumlah_Pencairan_Kotor', 'Saldo_Kotor', 'Jumlah_Penghapusan_Pokok']
for col in currency_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

# 3. Setelah selesai, baru bersihkan nama kolom untuk konsistensi
print("Membersihkan nama kolom...")
df = clean_col_names(df)

# 4. Definisikan fitur dan target menggunakan nama yang sudah bersih
target = 'StatusKreditMacet'
y = df[target]

# Pilih fitur yang relevan untuk model
numeric_features = df.select_dtypes(include=np.number).columns.tolist()
categorical_features = ['Provinsi']

# Hapus kolom yang tidak relevan/redundant
cols_to_remove = ['StatusKreditMacet', 'LoanNrChkDgt', 'KodePos', 'TahunFiskalPersetujuan', 'TahunPersetujuan']
numeric_features = [col for col in numeric_features if col not in cols_to_remove]

X = df[numeric_features + categorical_features]

# Lakukan pembagian data (NaN masih ada di dalam X, dan itu tidak apa-apa karena akan ditangani oleh pipeline)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print("Data siap untuk pelatihan model.")
print(f"Ukuran data latih: {X_train.shape}")
print(f"Ukuran data uji: {X_test.shape}")

Melakukan pra-pemrosesan pada kolom mata uang...
Membersihkan nama kolom...
Data siap untuk pelatihan model.
Ukuran data latih: (674373, 17)
Ukuran data uji: (224791, 17)


In [4]:
df.isna().sum()

,0
LoanNrChkDgt,0
NamaUMKM,14
Kota,30
NegaraBagianAS,14
KodePos,0
NamaBank,1559
NegaraBagianBank,1566
KodeIndustriNAICS,0
TanggalPersetujuan,0
TahunFiskalPersetujuan,0


# Bagian 2: Pencarian Model & Sampler Terbaik

In [5]:
# Pipeline untuk pra-pemrosesan fitur numerik
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Pipeline untuk pra-pemrosesan fitur kategorikal
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

# Gabungkan transformer ke dalam satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

models = {
    'RandomForest': RandomForestClassifier(random_state=42, class_weight='balanced'),
    'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', device='cuda'),
    'CatBoost': CatBoostClassifier(random_state=42, verbose=0, task_type='GPU')
}

samplers = {
    'TomekLinks': TomekLinks(sampling_strategy='auto'),
    'SMOTE': SMOTE(random_state=42),
    'None': 'passthrough'
}

best_score = -1
best_model_name = ""
best_pipeline = None

for model_name, model in models.items():
    for sampler_name, sampler in samplers.items():
        print(f"--- Menguji: Model [{model_name}] dengan Sampler [{sampler_name}] ---")
        pipeline = ImbPipeline(steps=[
            ('preprocessor', preprocessor),
            ('sampler', sampler),
            ('classifier', model)
        ])
        pipeline.fit(X_train, y_train)
        y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
        auprc = average_precision_score(y_test, y_pred_proba)
        print(f"Skor AUPRC: {auprc:.4f}")
        if auprc > best_score:
            best_score = auprc
            best_model_name = f"{model_name}_with_{sampler_name}"
            best_pipeline = pipeline

print(f"\n=== Model Terbaik Ditemukan ===")
print(f"Kombinasi: {best_model_name}")
print(f"Skor AUPRC Terbaik: {best_score:.4f}")

--- Menguji: Model [RandomForest] dengan Sampler [TomekLinks] ---
Skor AUPRC: 0.9814
--- Menguji: Model [RandomForest] dengan Sampler [SMOTE] ---
Skor AUPRC: 0.9838
--- Menguji: Model [RandomForest] dengan Sampler [None] ---
Skor AUPRC: 0.9843
--- Menguji: Model [XGBoost] dengan Sampler [TomekLinks] ---
Skor AUPRC: 0.9897
--- Menguji: Model [XGBoost] dengan Sampler [SMOTE] ---
Skor AUPRC: 0.9897
--- Menguji: Model [XGBoost] dengan Sampler [None] ---
Skor AUPRC: 0.9899
--- Menguji: Model [CatBoost] dengan Sampler [TomekLinks] ---
Skor AUPRC: 0.9905
--- Menguji: Model [CatBoost] dengan Sampler [SMOTE] ---
Skor AUPRC: 0.9902
--- Menguji: Model [CatBoost] dengan Sampler [None] ---
Skor AUPRC: 0.9909

=== Model Terbaik Ditemukan ===
Kombinasi: CatBoost_with_None
Skor AUPRC Terbaik: 0.9909


# Bagian 3: Penyimpanan Artefak Model Terbaik

In [6]:
joblib.dump(best_pipeline, 'best_model_pipeline.joblib')
print("Pipeline model terbaik disimpan sebagai 'best_model_pipeline.joblib'")

# Untuk membuat explainer SHAP, kita perlu data latih yang sudah ditransformasi
# Ambil langkah preprocessor dari pipeline terbaik dan transformasikan X_train
X_train_transformed = best_pipeline.named_steps['preprocessor'].transform(X_train)
model_classifier = best_pipeline.named_steps['classifier']

explainer = shap.TreeExplainer(model_classifier, X_train_transformed)

joblib.dump(explainer, 'best_explainer.joblib')
print("Explainer SHAP disimpan sebagai 'best_explainer.joblib'")

Pipeline model terbaik disimpan sebagai 'best_model_pipeline.joblib'
Explainer SHAP disimpan sebagai 'best_explainer.joblib'


# Bagian 4: Implementasi Explainable AI (XAI) untuk Transparansi Keputusan

In [7]:
def jelaskan_keputusan_kredit(data_umkm_baru):
    model_pipeline = joblib.load('best_model_pipeline.joblib')
    explainer = joblib.load('best_explainer.joblib')

    # Buat DataFrame dari input
    input_df = pd.DataFrame([data_umkm_baru])

    # Prediksi probabilitas gagal bayar
    pred_proba = model_pipeline.predict_proba(input_df)[0][1]

    if pred_proba > 0.5:
        prediction = "Kemungkinan Besar Ditolak"

        # Transformasi data input untuk mendapatkan SHAP values
        input_transformed = model_pipeline.named_steps['preprocessor'].transform(input_df)
        shap_values = explainer.shap_values(input_transformed)[0]
        feature_names = model_pipeline.named_steps['preprocessor'].get_feature_names_out()

        shap_df = pd.DataFrame([shap_values], columns=feature_names).T
        shap_df.columns = ['shap_value']
        shap_df['abs_shap'] = shap_df['shap_value'].abs()

        # Ambil 3-4 fitur pendorong penolakan teratas
        top_contributors = shap_df[shap_df['shap_value'] > 0].sort_values('abs_shap', ascending=False).head(4)

        penjelasan = f"Analisis Kredit: {prediction} (Probabilitas Gagal Bayar: {pred_proba:.1%})\n"
        penjelasan += "Berdasarkan analisis kami, ada beberapa faktor yang meningkatkan risiko kredit:\n"

        for feature, row in top_contributors.iterrows():
            clean_feature_name = re.sub(r'^(num__|cat)__', '', feature)

            if 'Provinsi' in clean_feature_name:
                provinsi_name = clean_feature_name.replace('Provinsi_', '')
                penjelasan += f"- Lokasi usaha di provinsi {provinsi_name.title()} menjadi salah satu faktor risiko utama.\n"
            else:
                # Cari nama kolom asli sebelum dibersihkan untuk mendapatkan nilai
                original_col_name = None
                for raw_col in data_umkm_baru.keys():
                    if re.sub(r'[^A-Za-z0-9]+', '', raw_col) == clean_feature_name:
                        original_col_name = raw_col
                        break
                value = data_umkm_baru.get(original_col_name, 'N/A')
                penjelasan += f"- {original_col_name.replace('_', ' ').title()} dengan nilai '{value}' teridentifikasi sebagai pendorong risiko signifikan.\n"
    else:
        prediction = "Kemungkinan Besar Disetujui"
        penjelasan = f"Analisis Kredit: {prediction} (Probabilitas Gagal Bayar: {pred_proba:.1%})\n"
        penjelasan += "Profil risiko peminjam dinilai cukup rendah berdasarkan data yang diberikan."

    return penjelasan